# Trawl processes simulations

In [1]:
from scipy.stats import levy_stable
import matplotlib.pyplot as plt
import numpy as np

The matrix `F` has shape (u-dim, k-dim, timesteps)

In [2]:
N = 300 # grid size
n = 100 # number of timesteps
M = 4 * N # extended grid size

gamma = 0.3
alpha = 1.9


The Hurst exponent $H$ equals:

In [3]:
(alpha-gamma)/alpha

0.8421052631578947

In [66]:
class TrawlProcess:
    def __init__(self, alpha, gamma, N=1000, n=100, M=5*N):
        self.H = (alpha-gamma)/alpha
        self.alpha = alpha
        self.gamma = gamma
        self.M = M
        self.N = N
        self.n = n
    
    def sample_paths(self, num_paths=1, plot=True):
        
        F = self._calculate_f()
        S = self._calculate_scale()
        
        stable_dist = levy_stable(alpha, 0.)
        draw = stable_dist.rvs(size=(num_paths, self.M, self.M))
        samples = draw * np.power(S, 1./self.alpha)
        samples = samples.reshape(-1, 1, self.M, self.M)
        time_samples =  samples * F
        paths = np.zeros((num_paths, self.n + 1))
        paths[:, 1:] = np.sum(time_samples, axis=(2,3))
        if plot:
            plt.style.use('seaborn-dark')
            fig, ax = plt.subplots(1,1, figsize=(8,8))
            ax.set_xlim(0,1)
            ax.grid()
            ax.legend(["H = " + str(self.H)])
            for path in range(paths.shape[0]):
                plt.plot(np.linspace(0,1, self.n+1), paths[path])
            plt.legend(["H = " + str(np.around(self.H, decimals=3))])
            plt.show()
        return paths
        
    
    def _calculate_scale(self):
        S = np.zeros((self.M, self.M), dtype=np.float64)
        for J in range(self.M):
            if J != 0:
                S[:, J] = (1./(self.N**(2+self.gamma)*(1+self.gamma))) * (J**(-1-self.gamma) - (J+1)**(-1-self.gamma))
        return S
        
    def _calculate_f(self):
        F = np.zeros((self.M, self.M, self.n), dtype=np.float32)
        for timestep in range(self.n):
            # (K, J)
            mask1 =np.triu(np.ones((self.M, self.M), dtype=np.bool)) # J >= K
    
            mask2 = np.zeros((self.M, self.M),  dtype=np.bool)
            mask2[0: timestep*(self.N//self.n), :] = True # K < timestep * (N/n)
            mask3 = np.triu(np.ones((self.M, self.M), dtype=np.bool), -timestep*(self.N//self.n)) # K - timestep (N/n) <= J
            f = np.zeros((N,N), dtype=np.float64)
    
            m1 = mask1*mask2
            f1 = np.zeros((self.M,self.M), dtype=np.float64)
            # Get a matrix A1 with column values
            A = np.array(range(self.M))
            A = A.reshape(1,-1)
            B = np.ones((self.M,1))
            A1 = (B @ A) # A1.T[k,j] = k
            f1 = A1.T * m1
    
            m2 = (~mask1) * mask2 # J < K and 0 <= k < timestep* N/n
            f2 = m2 * A1 
    
            m3 = mask1 * (~mask2)
            f3 = m3 * timestep*(self.N//self.n)
    
            m4 = (~mask1) * (~mask2) * mask3
            f4 = (timestep * (self.N//self.n) + A1 - A1.T) * m4
            f = f1 + f2 + f3 + f4
            #print(f.shape)
            F[:, :, timestep] = f
        
        F = F.reshape(self.n, self.M, self.M)
        return F
   

In [69]:
trawl = TrawlProcess(1.9, 0.3)
trawl.H

0.8421052631578947

In [ ]:
%%time
paths = trawl.sample_paths(num_paths=1)
paths.shape